In [13]:
import sys, os
import importlib.metadata

# Ajusta esta ruta si tu carpeta no se llama exactamente bullet3-3.25
sys.path.append(os.path.abspath("bullet3-3.25"))

import pybullet as p
import pybullet_data
import gymnasium as gym
import numpy as np
import torch

# Mostrar versiones para confirmar instalación
print("✅ PyBullet", importlib.metadata.version("pybullet"))
print("✅ Gymnasium", gym.__version__)
print("✅ Torch", torch.__version__)


✅ PyBullet 3.2.7
✅ Gymnasium 1.1.1
✅ Torch 2.5.1+cu121


In [14]:
from gymnasium import spaces

class RobotCupEnv(gym.Env):
    """Brazo Panda que voltea un vaso cilíndrico (reward = tilt − step_penalty)."""
    metadata = {"render_modes": ["human", "direct"], "render_fps": 240}

    def __init__(self, render_mode="direct"):
        super().__init__()
        self.render_mode = render_mode
        self.client = p.connect(p.GUI if render_mode=="human" else p.DIRECT)

        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        self.time_step = 1/240
        p.setTimeStep(self.time_step, physicsClientId=self.client)
        p.setGravity(0,0,-9.8, physicsClientId=self.client)

        # acciona 7 juntas
        self.action_space = spaces.Box(
            low=np.array([-2.9]*7, dtype=np.float32),
            high=np.array([ 2.9]*7, dtype=np.float32),
        )
        low  = np.array([-3.2]*7 + [-1.0, -1.0, 0.0], dtype=np.float32)
        high = np.array([ 3.2]*7 + [ 1.0,  1.0, 1.0], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        self.max_steps   = 240*5
        self.step_counter = 0

    def reset(self, *, seed=None, options=None):
        p.resetSimulation(physicsClientId=self.client)
        p.setGravity(0,0,-9.8, physicsClientId=self.client)
        p.setTimeStep(self.time_step, physicsClientId=self.client)
        p.loadURDF("plane.urdf", physicsClientId=self.client)

        self.robot_id = p.loadURDF(
            "franka_panda/panda.urdf",
            basePosition=[0,0,0],
            useFixedBase=True,
            physicsClientId=self.client
        )
        col = p.createCollisionShape(p.GEOM_CYLINDER, radius=0.03, height=0.1,
                                     physicsClientId=self.client)
        vis = p.createVisualShape(p.GEOM_CYLINDER, radius=0.03, length=0.1,
                                  rgbaColor=[1,1,1,1],
                                  physicsClientId=self.client)
        x = self.np_random.uniform(-0.3, 0.3)  # rango más pequeño para facilitar
        self.cup_id = p.createMultiBody(
            baseMass=0.1,
            baseCollisionShapeIndex=col,
            baseVisualShapeIndex=vis,
            basePosition=[x,0,0.05],
            physicsClientId=self.client
        )

        self.step_counter = 0
        return self._get_obs(), {}

    def step(self, action):
        for i, tgt in enumerate(action):
            p.setJointMotorControl2(
                self.robot_id, jointIndex=i,
                controlMode=p.POSITION_CONTROL,
                targetPosition=tgt,
                physicsClientId=self.client
            )
        p.stepSimulation(physicsClientId=self.client)
        self.step_counter += 1

        obs    = self._get_obs()
        reward = self._compute_reward()
        done   = self.step_counter >= self.max_steps
        term   = done or (reward >= 0.99)  # tilt casi 90°
        return obs, reward, term, False, {}

    def _get_obs(self):
        js     = p.getJointStates(self.robot_id, list(range(7)),
                                  physicsClientId=self.client)
        angles = [s[0] for s in js]
        pos,_  = p.getBasePositionAndOrientation(self.cup_id,
                                                  physicsClientId=self.client)
        return np.array(angles + list(pos), dtype=np.float32)

    def _compute_reward(self):
        _, quat = p.getBasePositionAndOrientation(self.cup_id,
                                                  physicsClientId=self.client)
        roll, pitch, _ = p.getEulerFromQuaternion(quat)
        tilt = max(abs(roll), abs(pitch))
        r_tilt = np.clip(tilt/(np.pi/2), 0, 1)
        # penalización por paso
        return float(r_tilt - 0.002)

    def render(self): pass
    def close(self): p.disconnect(physicsClientId=self.client)


In [15]:
env = RobotCupEnv(render_mode="direct")
obs, _ = env.reset()
for _ in range(200):
    action = env.action_space.sample()
    obs, reward, term, trunc, info = env.step(action)
    if term or trunc:
        break
env.close()
print("✅ Test direct finalizado | Reward:", reward)


✅ Test direct finalizado | Reward: -0.0018926094014711011


In [16]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

# 4 entornos en paralelo en modo DIRECT
venv = DummyVecEnv([lambda: RobotCupEnv(render_mode="direct") for _ in range(4)])
venv = VecMonitor(venv)   # sólo para registrar ep_len_mean y ep_rew_mean

model = PPO(
    "MlpPolicy",
    venv,
    verbose=1,
    device="cpu",
    n_steps=256,
    learning_rate=1e-4,
    ent_coef=0.05,
    batch_size=64,
)

model.learn(total_timesteps=500_000)
model.save("ppo_robot_cup_no_norm")


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 27       |
|    ep_rew_mean     | 14.9     |
| time/              |          |
|    fps             | 668      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 1024     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 27           |
|    ep_rew_mean          | 14.9         |
| time/                   |              |
|    fps                  | 559          |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0043286807 |
|    clip_fraction        | 0.00937      |
|    clip_range           | 0.2          |
|    entropy_loss         | -9.94        |
|    explained_variance   | 0.0285       

In [17]:
# Evaluación sin normalización
from stable_baselines3 import PPO

model = PPO.load("ppo_robot_cup_no_norm", device="cpu")
env   = RobotCupEnv(render_mode="human")

obs, _       = env.reset()
total_reward = 0.0

for _ in range(500):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, trunc, _ = env.step(action)
    total_reward += reward
    if done or trunc:
        break

# Tilt final
pos, quat = p.getBasePositionAndOrientation(env.cup_id, physicsClientId=env.client)
roll, pitch, _ = p.getEulerFromQuaternion(quat)

env.close()
print(f"🎯 Recompensa acumulada: {total_reward:.3f}")
print(f"🔄 Tilt final → roll: {roll:.2f} rad, pitch: {pitch:.2f} rad")


🎯 Recompensa acumulada: 14.565
🔄 Tilt final → roll: -1.57 rad, pitch: 0.91 rad
